In [ ]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'h2_dashboard.settings')
django.setup()

In [2]:
from django.apps import apps

def get_benchmark_data():
    # List of benchmark models you have, e.g., ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_models = ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_data = {}

    for model_name in benchmark_models:
        model = apps.get_model('h2_prod', model_name)  # Adjust 'h2_prod' to your app name if different
        benchmarks = model.objects.all()

        for benchmark in benchmarks:
            key = f"{model_name}-{benchmark.year}"
            if key not in benchmark_data:
                benchmark_data[key] = []
            benchmark_data[key].append(benchmark.value)

    return benchmark_data


In [5]:
def current_total_co2_emissions(total_h2_production):
    # Percentage of hydrogen production from each source
    percentage_gas = 0.48
    percentage_oil = 0.3
    percentage_coal = 0.18
    percentage_electro = 0.04

    # CO2 emissions (kg) from each source
    co2_per_kg_gas = 12  # kg CO2e per kg H2
    co2_per_kg_oil = 18  # kg CO2e per kg H2
    co2_per_kg_coal = 24  # kg CO2e per kg H2
    co2_per_kg_electro = 22  # kg CO2e per kg H2

    # Convert total hydrogen production from megatonnes to kilograms (1 megatonne = 1,000,000,000 kg)
    total_h2_prod_kg = total_h2_production * 1e9

    # Calculating the CO2 emissions from each source
    co2_emissions_gas = total_h2_prod_kg * percentage_gas * co2_per_kg_gas
    co2_emissions_oil = total_h2_prod_kg * percentage_oil * co2_per_kg_oil
    co2_emissions_coal = total_h2_prod_kg * percentage_coal * co2_per_kg_coal
    co2_emissions_electro = total_h2_prod_kg * percentage_electro * co2_per_kg_electro

    # Summing up the CO2 emissions from all sources to get the total CO2 emissions from hydrogen production
    previous_co2_emission = co2_emissions_gas + co2_emissions_oil + co2_emissions_coal + co2_emissions_electro
    return previous_co2_emission * 1e-9  # Conversion to Mega tonnes

print(round(current_total_co2_emissions(120)))

1963


Customize Your Templates:

Begin integrating the AdminLTE template into your Django views. Replace the HTML structure in your Django templates with the structure from AdminLTE's HTML files, making sure to keep Django template tags and context variables intac